In [20]:
lines=[]
count=0
#Max=-1
Max=10000
with open('train.txt','r') as f:
    for s in f:
        count+=1
        if count>Max and Max!=-1:
            break
        lines.append(s.strip())
#print(len(lines))
print(lines[:10])       


["Here are two reasons companies fail: they only do more of the same, or they only do what's new", 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation', 'Both are necessary, but it can be too much of a good thing', 'Consider Facit', "I'm actually old enough to remember them", 'Facit was a fantastic company', 'Everybody used them', 'And what did Facit do when the electronic calculator came along? They continued doing exactly the same', 'In six months, they went from maximum revenue', 'and they were gone']


In [21]:
# tokenize sentences 
import nltk
sentences=[]
all_tokens_count=0
for line in lines:
    tokens = nltk.word_tokenize(line.lower())
    all_tokens_count+=len(tokens)
    #sentences.append(tokens)
    sentences.append(['<s>']+tokens+['</s>'])
print('all_tokens_count =',all_tokens_count)
print(len(sentences))
print(sentences[:5])


all_tokens_count = 185925
10000
[['<s>', 'here', 'are', 'two', 'reasons', 'companies', 'fail', ':', 'they', 'only', 'do', 'more', 'of', 'the', 'same', ',', 'or', 'they', 'only', 'do', 'what', "'s", 'new', '</s>'], ['<s>', 'to', 'me', 'the', 'real', ',', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', ':', 'exploration', 'and', 'exploitation', '</s>'], ['<s>', 'both', 'are', 'necessary', ',', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing', '</s>'], ['<s>', 'consider', 'facit', '</s>'], ['<s>', 'i', "'m", 'actually', 'old', 'enough', 'to', 'remember', 'them', '</s>']]


In [22]:
# counting 1-gram 
from collections import Counter
counter_unigram=Counter()
for sent in sentences:
    counter_unigram.update(sent)
V=len(counter_unigram)
#print(counter_unigram)
print('V =',V)
n=0
for gram in counter_unigram:
    n+=counter_unigram[gram]
n=n-counter_unigram['<s>']-counter_unigram['</s>']
print('n =',n)

V = 12838
n = 185925


# CALCULATE N1,N2,N3,N4 OF GOOD TURING

In [31]:
N1 = list(counter_unigram.values()).count(1)
print(N1)

5971


In [24]:
N2 = list(counter_unigram.values()).count(2)
print(N2)

2006


In [25]:
N3 = list(counter_unigram.values()).count(3)
print(N3)

1010


In [28]:
N4 = list(counter_unigram.values()).count(4)
print(N4)

645


In [29]:
from nltk import ngrams
bi_grams=[]
for sent in sentences:
    gram2=ngrams(sent,2)
    bi_grams.extend(gram2)
print(len(bi_grams))

for i in range(3):
    print(bi_grams[i])

counter_bigram = Counter(bi_grams)
print('V =',len(counter_bigram))
print(counter_bigram[('here','are')])

195925
('<s>', 'here')
('here', 'are')
('are', 'two')
V = 80120
10


In [5]:
tri_grams=[]
for sent in sentences:
    gram3=ngrams(sent,3)
    tri_grams.extend(gram3)
print(len(tri_grams))

for i in range(3):
    print(tri_grams[i])

counter_trigram = Counter(tri_grams)
print('V=',len(counter_trigram))
print(counter_trigram[('here','are','two')])

185925
('<s>', 'here', 'are')
('here', 'are', 'two')
('are', 'two', 'reasons')
V= 142550
2


In [6]:
four_grams=[]
for sent in sentences:
    gram4=ngrams(sent,4)
    four_grams.extend(gram4)
print(len(four_grams))

for i in range(3):
    print(four_grams[i])
counter_fourgram = Counter(four_grams)
print('V =',len(counter_fourgram))
print(counter_fourgram[('here','are','two','reasons')])

175929
('<s>', 'here', 'are', 'two')
('here', 'are', 'two', 'reasons')
('are', 'two', 'reasons', 'companies')
V = 163392
1


In [7]:
# # tính theo laplace
# alpha = 0.001
# def uni_prob(word):
#     return max(1,counter_unigram[word]+alpha)/all_token_count
# def bi_prob(word1,word2):
#     return (counter_bigram[(word1,word2)]+alpha)/(counter_unigram[word1]+V)
# def tri_prob(word1,word2,word3):
#     return (counter_trigram[(word1,word2,word3)]+alpha)/(counter_bigram[(word1,word2)]+1+V)

In [8]:
# backoff
# tính prob theo từng loại: 1-gram, 2-gram, 3-gram
def uni_prob(word):
    return max(1,counter_unigram[word])/all_tokens_count
print(uni_prob('here'))
def bi_prob(word1,word2):
    if counter_bigram[(word1,word2)]>0:
        return counter_bigram[(word1,word2)]/counter_unigram[word1]
    else:
        return 0.4*uni_prob(word2)
print(bi_prob('here','are'))    
def tri_prob(word1,word2,word3):
    if counter_trigram[(word1,word2,word3)]>0:
        return counter_trigram[(word1,word2,word3)]/counter_bigram[(word1,word2)]
    else:
        return 0.4*bi_prob(word1,word2)
print(tri_prob('here','are','two'))
def four_prob(word1,word2,word3,word4):
    if counter_fourgram[(word1,word2,word3,word4)]>0:
        return counter_fourgram[(word1,word2,word3,word4)]/counter_trigram[(word1,word2,word3)]
    else:
        return 0.4*tri_prob(word1,word2,word3)
print(four_prob('here','are','two','reasons'))

0.0013553852359822509
0.03968253968253968
0.2
0.5


In [9]:
# tính xác suất của một câu, normalize theo 1 từ 
def probLM(sent,n):
    if n>4 or n<1: # không xét trường hợp này 
        return 0
    tokens=nltk.word_tokenize(sent.lower())
    tokens += ['<s>']+tokens
    prob=1
    for i in range(1,len(tokens)):
        if n==1:
            prob*=uni_prob(tokens[i])
        elif n==2:
            prob*=bi_prob(tokens[i-1],tokens[i])
        elif n==3:
            if i>=2:
                prob*=tri_prob(tokens[i-2],tokens[i-1],tokens[i])
            else:
                prob*=bi_prob(tokens[i-1],tokens[i])
    l=len(tokens)-1
    return prob**(1/l)

In [10]:
sent='the human body with new abilities is no longer a question'
#sent='the human body with new from abilities is no longer a question'
#sent='A few years back from'
print('n=1')
pr=probLM(sent,1)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=2')
pr=probLM(sent,2)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=3')
pr=probLM(sent,3)
print('prob=',pr)
print('perplexity=',1/pr)

n=1
prob= 0.0014115277343780864
perplexity= 708.4522504551397
n=2
prob= 0.01850971039549826
perplexity= 54.02569670907491
n=3
prob= 0.1577713762609277
perplexity= 6.338285332227602


In [11]:
# kiểm tra xem 2 xâu có xác suất hơn nhau thế nào, ví dụ cho bài toán speech to text
sent1='the human body with new abilities is no longer a question'
sent2='the human body with knew abilities is know longer a question'
pr=probLM(sent1,3)
print('prob=',pr)
print('perplexity=',1/pr)

pr=probLM(sent2,3)
print('prob=',pr)
print('perplexity=',1/pr)

prob= 0.1577713762609277
perplexity= 6.338285332227602
prob= 0.00391831904286787
perplexity= 255.21147947873246


In [12]:
w1 = 'here'
w2 = 'are'
w3 = 'two'
def get_word(w1,w2,w3):
    p = 0
    word =''
    for w in counter_unigram.keys():
        if p<four_prob(w1,w2,w3,w):
            p = four_prob(w1,w2,w3,w)
            word = w
    return word
print(get_word(w1,w2,w3))


reasons


In [13]:
def get_sent(w1,w2,w3,n):
    sent=[w1,w2,w3]
    while n>0:
        for i in range(n):
            sent.append(get_word(sent[i],sent[i+1],sent[i+2]))
            n-=1
    return sent
get_sent(w1,w2,w3,5)


['here', 'are', 'two', 'reasons', 'companies', 'fail', ':', 'they']

In [25]:
sentences = 'Exploitation is taking the knowledge we have and making good, better\
 Exploitation is about making our trains run on time\
 It\'s about making good products faster and cheaper\
 Exploitation is not risky -- in the short term\
 But if we only exploit, it\'s very risky in the long term\
 And I think we all have memories of the famous pop groups who keep singing the same songs again and again, until they become obsolete or even pathetic\
 So if we take a long-term perspective, we explore'
string = nltk.word_tokenize(sentences)
print(string)

['Exploitation', 'is', 'taking', 'the', 'knowledge', 'we', 'have', 'and', 'making', 'good', ',', 'better', 'Exploitation', 'is', 'about', 'making', 'our', 'trains', 'run', 'on', 'time', 'It', "'s", 'about', 'making', 'good', 'products', 'faster', 'and', 'cheaper', 'Exploitation', 'is', 'not', 'risky', '--', 'in', 'the', 'short', 'term', 'But', 'if', 'we', 'only', 'exploit', ',', 'it', "'s", 'very', 'risky', 'in', 'the', 'long', 'term', 'And', 'I', 'think', 'we', 'all', 'have', 'memories', 'of', 'the', 'famous', 'pop', 'groups', 'who', 'keep', 'singing', 'the', 'same', 'songs', 'again', 'and', 'again', ',', 'until', 'they', 'become', 'obsolete', 'or', 'even', 'pathetic', 'So', 'if', 'we', 'take', 'a', 'long-term', 'perspective', ',', 'we', 'explore']
